In [ ]:
pip install "pycozo[embedded,pandas]"

In [1]:
from pycozo.client import Client
import pandas as pd
from pycozo.client import Client
client = Client('rocksdb', 'datafile.db')

In [5]:
# Create the model structure 
res = client.run('{:create node {id_n: Int}} {:create edge {id_e: Int, ns: Int, nd: Int}} {:create label {ln: String}} {:create prop {key: String}} {:create node_label {id_n: Int, ln: String}} {:create edge_label {id_e: Int, ln: String}} {:create node_prop {id_n: Int, key: String, value}} {:create edge_prop {id_e: Int, key: String, value}}')
res

,status
0,OK


In [ ]:
# Add indexes
res = client.run('{::index create node:id_idx {id_n}} {::index create edge:id_idx {id_e, ns, nd}} {::index create node_label:id_idx {id_n}} {::index create edge_label:id_idx {id_e, ln}} {::index create node_prop:id_idx {id_n, key}} {::index create edge_prop:id_idx {id_e, key}}')
    
print(res)

In [ ]:
# check the existing cozo relations for every graph element
res = client.run('::relations')
res

In [ ]:
# query the graph
res = client.run('?[ id_n, ln] := *node_label[ id_n, ln]')
res

In [2]:
# from Benchmark to  Cozo (insert data in node[id_n])
# we divided the whole benchmark to 30 datasets, each dataset contains 100K nodes & 500K edges
#/!\ DO NOT FORGET TO CHANGE THE STRAT & END ids if you want to have multiple datasets

def generate_cozo_node(existing_max_id=0, final_max_id=100000):

    start = existing_max_id + 1
    end   = final_max_id
    # Construire la liste des IDs
    node_lines = ",".join(f"[{i}]" for i in range(start, end + 1))
    # Construire la commande Cozo
    node_cmd = f"?[id_n] <- [{node_lines}] :put node {{id_n}}"
    return node_cmd

cmd = generate_cozo_node(existing_max_id=0, final_max_id=100000)
# print(cmd)
res = client.run(cmd)

res

,status
0,OK


In [ ]:
#  check the inserted data 
res = client.run('?[ id_n] := *node[ id_n]')
res

In [3]:
# from Benchmark to  Cozo (insert data in node_label[id_n, ln])
# we divided the whole benchmark to 30 datasets, each dataset contains 100K nodes & 500K edges
#/!\ DO NOT FORGET TO CHANGE THE STRAT & END ids if you want to have multiple datasets
import random
random.seed(12)
node_labels = [
    "forum",
    "person",
    "place",
    "post",
    "organisation",
    "comment",
    "emailaddress",
    "language",
    "tagclass",
    "tag"
]

def generate_node_label_cozo_range(start_id=1, end_id=100000):
    """
    Génère la règle Cozo pour assigner aléatoirement un label
    à chaque nœud dont l'ID va de start_id à end_id inclus.
    """
    assignments = []
    for node_id in range(start_id, end_id + 1):
        label = random.choice(node_labels)
        assignments.append(f"[{node_id}, \"{label}\"]")
    assignments_str = ",".join(assignments)
    return f"?[id_n, ln] <- [{assignments_str}] :put node_label {{id_n, ln}}"

# call
cmd2 = generate_node_label_cozo_range(1, 100000)
# print(cmd2)
res = client.run(cmd2)
res

,status
0,OK


In [10]:
# from Benchmark to  Cozo (insert data in edge_label[id_e, ln])
# we divided the whole benchmark to 30 datasets, each dataset contains 100K nodes & 500K edges
#/!\ DO NOT FORGET TO CHANGE THE STRAT & END ids if you want to have multiple datasets
import random
random.seed(23)
edge_labels = [
    "hasMemberWithPosts",
    "hasCreator",
    "isLocatedin",
    "hasTag",
    "replyOf",
    "isPartOf",
    "containerOf",
    "hasMember",
    "hasModerator",
    "likes",
    "hasInterest",
    "knows",
    "speaks",
    "studyAt",
    "workAt",
    "hasType",
    "isSubclassOf",
    "email"
]

def generate_edge_label_cozo_range(start_id=1, end_id= 500_000):
    """
    Génère la règle Cozo pour assigner un label aléatoire
    à chaque arête dont l'ID va de start_id à end_id inclus.
    """
    assignments = []
    for edge_id in range(start_id, end_id + 1):
        label = random.choice(edge_labels)
        assignments.append(f"[{edge_id}, \"{label}\"]")
    assignments_str = ",".join(assignments)
    return f"?[id_e, ln] <- [{assignments_str}] :put edge_label {{id_e, ln}}"

# Usage
cmd3 = generate_edge_label_cozo_range(1, 500_000)


res = client.run(cmd3)
res

,status
0,OK


In [ ]:

# from Benchmark to  Cozo (insert data in node_prop[id_n, key, value])
# we divided the whole benchmark to 30 datasets, each dataset contains 100K nodes & 500K edges
#/!\ DO NOT FORGET TO CHANGE THE STRAT & END ids if you want to have multiple datasets
import random
from datetime import datetime, timedelta
import pandas as pd

# ── Set the seed for reproducibility──────
random.seed(4)

# ── random dates  ───────────────────────────────────────
def random_date(start_year=1980, end_year=2020):
    start = datetime(start_year,1,1)
    end   = datetime(end_year,12,31)
    delta = (end - start).days
    return (start + timedelta(days=random.randint(0,delta))).isoformat()

# ── properties per label ─────────────────────────────────
label_properties = {
    "person":       ["firstName","lastName","gender","birthday","creationDate","locationIP","browserUsed"],
    "place":        ["type","name","url"],
    "organisation": ["type","name","url"],
    "post":         ["imageFile","creationDate","locationIP","browserUsed","content","length"],
    "comment":      ["creationDate","locationIP","browserUsed","content","length"],
    "forum":        ["title","creationDate"],
    "tag":          ["name","url"],
    "tagclass":     ["name","url"],
    "emailaddress": ["email"],
    "language":     ["name"]
}

# ── name values up2 label/type ──────────────────────
organisation_names = {
    "University": [
    "MIT",
    "Stanford",
    "Cambridge",
    "Oxford",
    "Harvard",
    "Caltech",
    "Princeton",
    "Yale",
    "Columbia",
    "University of Chicago",
    "UC Berkeley",
    "ETH Zurich",
    "University of Tokyo",
    "Tsinghua University",
    "Peking University",
    "Imperial College London",
    "UCL",
    "University of Toronto",
    "University of Melbourne",
    "National University of Singapore",
    "University of Edinburgh",
    "Cornell University",
    "University of Michigan",
    "Duke University",
    "University of Pennsylvania",
    "Carnegie Mellon University",
    "Johns Hopkins University",
    "Northwestern University",
    "University of California, Los Angeles",
    "University of British Columbia",
    "Seoul National University",
    "KAIST",
    "Australian National University",
    "University of Sydney",
    "University of Manchester",
   
],
    "Company":     ["Powtentia", "Apple", "Intel",
    "IBM",
    "Oracle",
    "Salesforce",
    "Adobe",
    "Spotify",
     "LinkedIn",
    "Dropbox",
    "Slack",
    "Pinterest",
    "PayPal",
    "Shopify",
    "Square",
    "Atlassian",
    "GitHub",
    "Reddit"
    
]
}
place_names = {
    "Country": [
    "France",
    "USA",
    "Japan",
    "Germany",
    "Australia",
    "Canada",
    "United Kingdom",
    "China",
    "India",
    "Brazil",
    "Italy",
    "Spain",
    "Netherlands",
    "South Korea",
    "Russia",
    "Mexico",
    "Sweden",
    "Switzerland",
    "Singapore"
]
,
    "City":    [
    "Paris",
    "New York",
    "Tokyo",
    "Berlin",
    "Sydney",
    "London",
    "San Francisco",
    "Los Angeles",
    "Chicago",
    "Toronto",
    "Melbourne",
    "Barcelona",
    "Amsterdam",
    "Seoul",
    "Shanghai",
    "Moscow",
    "Dubai",
    "Singapore",
    "Vancouver",
    "Hong Kong"
]

}
tag_names = [
    "Tech",
    "Productivity",
    "Coding",
    "Design",
    "RemoteWork",
    "Motivation",
    "Startups",
    "Learning",
    "JavaScript",
    "React",
    "Career",
    "Freelance",
    "AI",
    "UX",
    "Lifestyle",
    "Tools",
    "OpenSource",
    "BugFix",
    "Tutorial",
    "Networking",
    "WorkLifeBalance",
    "Innovation",
    "Tips",
    "Resources",
    "Community",
    "Debugging",
    "Software",
    "Health",
    "Travel",
    "Inspiration",
    "Projects",
    "Collaboration",
    "Events",
    "Announcements",
    "Questions",
    "Feedback",
    "Goals",
    "Books",
    "Podcast",
    "Music",
    "Hustle",
    "ProductLaunch"
]

tagclass_names = ["Technology","Science","Art","History","Music"]
language_names = ["English","French","Spanish","German","Chinese"]

# ── Textes spécifiques pour content ───────────────────────────────────────
content_values = {
    "post": [
    "Check out our new blog post!",
    "Today I learned about property graphs.",
    "Here is a photo from my trip.",
    "Launching our new product tomorrow!",
    "Just finished reading a fantastic book.",
    "Coffee is life. ☕",
    "Working on something exciting. Stay tuned!",
    "Throwback to last summer.",
    "Big update coming next week!",
    "Finally solved that bug 🐛",
    "Weekend vibes 🌴",
    "We just hit 10,000 users!",
    "New tutorial just dropped on the blog.",
    "Exploring the mountains today.",
    "Anyone else love rainy days?",
    "Trying out a new recipe tonight.",
    "Productivity tip: block your mornings.",
    "Sunsets never get old.",
    "Version 2.0 is live now!",
    "Woke up early and crushed my to-do list.",
    "Happy Friday everyone!",
    "Deep dive into graph databases coming soon.",
    "Grateful for this amazing team.",
    "Behind the scenes of our latest shoot.",
    "New gear arrived today!",
    "Just published a new open source library.",
    "Working from the beach today 🌊",
    "Our roadmap for Q3 is here.",
    "First snowfall of the year!",
    "Just met someone who uses our app daily 🤯",
    "Beta testers wanted!",
    "Early morning run = best part of my day.",
    "Minimalism is underrated.",
    "Sketching out some new UI ideas.",
    "Monday motivation: keep showing up.",
    "Live Q&A tomorrow at 6pm!",
    "This view never gets old.",
    "Launching a podcast soon!",
    "Hackathon weekend starts now.",
    "Big changes are coming.",
    "Learning Rust and loving it.",
    "Who else loves clean code?",
    "Your feedback means everything to us.",
    "Got featured in a tech newsletter!",
    "Daily design inspiration 💡",
    "Finally built dark mode!",
    "This community is incredible.",
    "Cant stop listening to this song.",
    "Just crossed 1M API calls!",
    "Working on speed optimizations today.",
    "Books > TV. Always.",
    "Clouds looked like cotton candy today ☁️",
    "Back to the grind. Lets go!",
    "One step closer to launch.",
    "Testing out some crazy UI animations.",
    "Almost there... just one more bug.",
    "Our new branding is live!",
    "Who wants early access?",
    "Just deployed a hotfix.",
    "Feeling inspired today.",
    "Design sprint was a success!",
    "Guess what we’re working on? 🔍",
    "Late-night coding sessions hit different.",
    "Grabbing a matcha and hitting the keyboard.",
    "Rewatching Interstellar. Again.",
    "New city. New energy.",
    "Big milestone unlocked!",
    "Planning our next feature drop.",
    "Just found a killer productivity app.",
    "Remember to take breaks.",
    "Code. Sleep. Repeat.",
    "What a wild year its been.",
    "Thanks for 5 years of support!",
    "Traveling light, thinking deep.",
    "Who else loves keyboard shortcuts?",
    "Dark themes FTW.",
    "Just added search functionality!",
    "Building your own tools is underrated.",
    "Started journaling again.",
    "UX tweaks coming in the next release.",
    "Who is hiring? Looking for my next adventure.",
    "What podcasts are you into lately?",
    "Just hit inbox zero! 🎉",
    "Why is naming things so hard?",
    "Refactoring old code is oddly satisfying.",
    "Join us for a live demo tomorrow.",
    "Brainstorming with the team today.",
    "Back on my workout routine!",
    "Loving the new dev tools update.",
    "Time to unplug for the weekend.",
    "Feeling grateful today.",
    "Creating mockups all afternoon.",
    "Whatis your favorite VS Code extension?",
    "Here is a peek at our latest prototype.",
    "Trying out a 4-day workweek.",
    "Launching our new site tonight!",
    "Midweek check-in: how everyone doing?",
    "Pizza + code = perfect night.",
    "Redesigning our onboarding flow.",
    "Just passed 100 stars on GitHub!",
    "Reflections from today standup.",
    "Stuck on a problem? Go for a walk."
]
,
    "comment": [
    "Great article",
    "I totally agree with this",
    "Can you explain more",
    "Thanks for sharing",
    "This is so helpful, thanks",
    "Interesting perspective",
    "I hadnt thought of it that way",
    "Couldnt have said it better myself",
    "Love this",
    "Thanks for the insight",
    "Really well written",
    "Super informative",
    "Bookmarking this for later",
    "I learned something new today",
    "Great point",
    "This makes a lot of sense",
    "Do you have any sources for this",
    "Helpful and concise",
    "Nice breakdown",
    "Thanks for clarifying 👍",
    "Amazing content, keep it up",
    "This helped me so much",
    "Totally agree",
    "Ive been wondering about this",
    "You explained this really clearly",
    "Mind blown 🤯",
    "This deserves more attention",
    "Sharing this with my team",
    "Where can I learn more",
    "I had no idea — thanks"
]

}

# ── Générateur de valeur selon clé & contexte ───────────────────────────
def generate_value(key, label, current_props):
    # 1) type→name pour organisation / place
    if key == "type" and label=="organisation":
        return random.choice(["University","Company"])
    if key == "type" and label=="place":
        return random.choice(["Country","City"])
    if key == "name":
        if label=="organisation":
            typ = current_props["type"]
            return random.choice(organisation_names[typ])
        if label=="place":
            typ = current_props["type"]
            return random.choice(place_names[typ])
        if label=="tag":
            return random.choice(tag_names)
        if label=="tagclass":
            return random.choice(tagclass_names)
        if label=="language":
            return random.choice(language_names)
        # fallback générique
        return "Unknown"
    # 2) content spécifique
    if key=="content":
        return random.choice(content_values[label])
    # 3) autres clés
    if key=="firstName":    return random.choice( ['Mahinda', 'Carmen', 'Rahul', 'Albin', 'Meera', 'A.', 'Jack', 'Gyorgy', 'Eric', 'Chris', 'Gustavo', 'Elmar', 'Yang', 'Eli', 'Ida', 'Peter', 'Alfred', 'Babar', 'Ramon', 'Antonio', 'Jie', 'Arjun', 'George', 'Eva', 'Michael', 'Natalia', 'Claude', 'Bill', 'Francisco', 'Joseph', 'Gabriel', 'Karl', 'Matias', 'Avraham', 'Jan', 'Gamini', 'Chen', 'Seung-Won', 'Gunadasa', 'Chau', 'Hao', 'Albade', 'Jun', 'Charles', 'Ge', 'Boy', 'Wei', 'Eduardo', 'Manuel', 'Andrej', 'Priyanka', 'Fengyi', 'Jerzy', 'Yacine', 'Faisal', 'Ning', 'Jacqueline', 'Carlos', 'Abdul Rahman', 'Imtiaz', 'Alexei', 'David', 'Muhammad', 'Badruddin', 'Jose', 'Kamel', 'Seyni'])
    if key=="lastName":     return random.choice(['Perera', 'Lepland', 'Do', 'Kumar', 'Reddy', 'Monteno', 'Rao', 'Smith', 'Kiss', 'Law-Yone', 'Jasani', 'Arbelaez', 'Andrle', 'Wang', 'Peretz', 'Baldini', 'Taylor', 'Mol', 'Ali', 'Sotto', 'Gonzales', 'Chen', 'Khan', 'King', 'Filipensky', 'Jones', 'Hleb', 'Muni', 'Brown', 'Martins', 'Anderson', 'Rodriguez', 'Becker', 'Martinez', 'Al-Shorabi', 'Beran', 'Herath', 'Zhou', 'Lee', 'Loan', 'Li', 'Maazou', 'Calvert', 'Zhang', 'Aquino', 'Yang', 'Pinto', 'Silva', 'Benedejcic', 'Froome', 'Xu', 'Ciesla', 'Abdelli', 'Malik', 'Ha', 'Alvarez', 'Budjana', 'Ahmed', 'Berman', 'Baker', 'Gonzi', 'Gomez', 'Omar', 'Alonso', 'Abderrahmane', 'Kountche'])
    if key=="gender":       return random.choice(["male","female","other"])
    if key=="birthday":     return random_date(1950,2005)
    if key=="creationDate": return random_date(2008,2022)
    if key=="locationIP":   
        return f"{random.randint(1,255)}.{random.randint(0,255)}.{random.randint(0,255)}.{random.randint(1,255)}"
    if key=="browserUsed":  return random.choice(["Chrome","Firefox","Safari","Internet Explorer"])
    if key=="url":          return f"http://example.com/{random.randint(1000,9999)}"
    if key=="length":       return str(random.randint(10,300))
    if key=="imageFile":    return random.choice(["photo1.jpg","pic.png","img.jpeg"])
    if key=="title":        return random.choice(["Need help", "Bug report", "Quick question", "UI feedback", "Feature request", "New project", "Just launched", "Looking for tips", "Daily update", "First post", "New tool", "Code review", "Any advice", "Best resources", "Quick win", "Design ideas", "Progress update", "Seeking feedback", "Why this matters", "Motivation needed", "Tool comparison", "Getting started", "Need suggestions", "Tips and tricks", "Funny story", "Coding goals", "Tiny update", "Dev journey", "Release notes", "Next steps"])
    if key=="email":        return random.choice(["user1@example.com","test@mail.com"])
    return "unknown"

# ── Fonction centrale générant uniquement les règles Cozo ─────────────────
def generate_node_prop_cozo_only(node_label_df, batch_size=500):
    cozo_entries = []
    
    for _, row in node_label_df.iterrows():
        id_n  = row["id_n"]
        label = row["ln"]
        props = label_properties.get(label, [])
        current_props = {"id": id_n}
        # parcours ordonné pour gérer type→name
        for key in props:
            val = generate_value(key, label, current_props)
            current_props[key] = val
            cozo_entries.append(f"[{id_n},\"{key}\",\"{val}\"]")
    
    # découpe en batches et formulation des règles Cozo
    cozo_scripts = []
    for i in range(0, len(cozo_entries), batch_size):
        batch = ",".join(cozo_entries[i:i+batch_size])
        cozo_scripts.append(
            f"?[id_n,key,value] <- [{batch}] :put node_prop {{id_n,key,value}}"
        )
    return cozo_scripts

# ── Execution ────────────────────────────────────────────────────────────
# 1) Load & filter node_label_df to add node_prop only to the new added nodes in each dataset
node_label_df = pd.DataFrame(client.run('?[id_n,ln] := *node_label[id_n,ln]'))
node_label_df = node_label_df[(node_label_df["id_n"] >= 1) &(node_label_df["id_n"] <= 100000)]

# 2)  Cozo rules
cozo_rules = generate_node_prop_cozo_only(node_label_df, batch_size=20)

# 3) Execute cozo rules
for rule in cozo_rules:
     res = client.run(rule)
     print(res)

# print(cozo_rules[0])


In [ ]:
# check
res = client.run('?[n, k, v] := *node_prop[n, k,v]')

res

In [11]:
# from Benchmark to  Cozo (insert data in  edge[id_e, ns, nd])
# we divided the whole benchmark to 30 datasets, each dataset contains 100K nodes & 500K edges
#/!\ DO NOT FORGET TO CHANGE THE STRAT & END ids if you want to have multiple datasets

import random
import pandas as pd
from collections import defaultdict

# ── Pour la reproductibilité ─────────────────────────────────────────────
random.seed(14)

# ── Charger les labels depuis CozoDB ────────────────────────────────────
node_label = pd.DataFrame(client.run('?[id_n,ln] := *node_label[id_n,ln]'))
node_label_df = node_label[(node_label["id_n"] >= 1) &(node_label["id_n"] <= 500000)] 
                                    #have different filter bcoz of different size
edge_label = pd.DataFrame(client.run('?[id_e,ln] := *edge_label[id_e,ln]'))

edge_label_df = edge_label[(edge_label["id_e"] >= 1) &(edge_label["id_e"] <= 500000)]

# ── Définir les patterns de relation SNB ────────────────────────────────
relation_patterns = [
    "forum_hasMemberWithPosts_person", "comment_hasCreator_person",
    "organisation_isLocatedin_place", 
    "post_isLocatedIn_place", "comment_isLocatedin_place",
    "comment_hasTag_tag", "comment_replyOf_comment",
    "comment_replyOf_post", "place_isPartOf_place",
    "forum_containerOf_post", "forum_hasMember_person",
    "forum_hasModerator_person", "forum_hasTag_tag",
    "person_likes_comment","person_likes_post", "person_workAt_organisation",
    "person_studyAt_organisation", "person_speaks_language", "person_knows_person",
    "person_isLocatedIn_place", "person_hasInterest_tag", "person_email_emailaddress",
    "post_hasCreator_person","post_hasTag_tag",
    "tagclass_isSubclassOf_tagclass","tag_hasType_tagclass"
]

# mapping edgelabel → liste de (ns_label, nd_label)
label_to_relation = defaultdict(list)
for rel in relation_patterns:
    ns, lbl, nd = rel.split("_", 2)
    label_to_relation[lbl].append((ns, nd))

# mapping node_label → liste d’IDs
label_to_ids = node_label_df.groupby("ln")["id_n"].apply(list).to_dict()

cozo_entries = []

for _, row in edge_label_df.iterrows():
    id_e, elabel = row["id_e"], row["ln"]
    if elabel not in label_to_relation:
        continue

    # On prend **au hasard** UNE seule paire possible
    possible_pairs = label_to_relation[elabel]
    ns_lbl, nd_lbl = random.choice(possible_pairs)

    src_ids = label_to_ids.get(ns_lbl, [])
    dst_ids = label_to_ids.get(nd_lbl, [])
    if not src_ids or not dst_ids:
        continue

    src = random.choice(src_ids)
    dst = random.choice(dst_ids)
    cozo_entries.append(f"[{id_e},{src},{dst}]")
    # pas de break nécessaire, on a déjà choisi la paire

# Formater et exécuter
cmd = "?[id_e,ns,nd] <- [" + ",".join(cozo_entries) + "] :put edge {id_e,ns,nd}"
res = client.run(cmd)
print(res)

  status
0     OK


In [ ]:
# check
res = client.run('?[e, s,d] := *edge[e,s,d]')
res

In [12]:
# from Benchmark to  Cozo (insert data in  edge_prop[id_e, key, value])
# we divided the whole benchmark to 30 datasets, each dataset contains 100K nodes & 500K edges
#/!\ DO NOT FORGET TO CHANGE THE STRAT & END ids if you want to have multiple datasets
import random
from datetime import datetime, timedelta
import pandas as pd

random.seed(21)

edge_properties = {
    "hasMember":    ["joinDate"],
    "likes":        ["creationDate"],
    "knows":        ["since", "strength"],
    "hasModerator": ["assignedDate"],
    "studyAt":      ["classYear"],
    "workAt":       ["workFrom"],
    "replyOf":      ["confidence"],
    "hasTag":       ["relevance"],
    "isLocatedin":  ["certainty"]
}

# Pré-compiler les générateurs de valeurs pour chaque clé
def make_date_generator(start, end):
    delta = (end - start).days
    def gen():
        return (start + timedelta(days=random.randint(0, delta))).strftime("%Y-%m-%d")
    return gen

_gen_date = make_date_generator(datetime(2000,1,1), datetime(2024,12,31))

def gen_val(key):
    if key in ("joinDate","creationDate","assignedDate","since"):
        return _gen_date()
    if key == "classYear":
        return str(random.randint(2000,2025))
    if key == "workFrom":
        return str(random.randint(1990,2024))
    if key == "strength":
        return f"{random.uniform(0.1,1.0):.2f}"
    if key == "confidence":
        return str(random.randint(1,10))
    if key == "relevance":
        return str(random.randint(1,100))
    if key == "certainty":
        return random.choice(["high","medium","low"])
    return "N/A"

def generate_edge_prop_cozo(edge_label_df, batch_size=1000):
    # 1) Génération vectorisée de toutes les entrées
    cozo_entries = [
        f"[{row.id_e},\"{key}\",\"{gen_val(key)}\"]"
        for row in edge_label_df.itertuples()
        for key in edge_properties.get(row.ln, [])
    ]
    # 2) Découpage en batchs larges
    rules = []
    for i in range(0, len(cozo_entries), batch_size):
        batch = ",".join(cozo_entries[i:i+batch_size])
        rules.append(f"?[id_e,key,value] <- [{batch}] :put edge_prop {{id_e,key,value}}")
    return rules

# === appel ===
edge_label = pd.DataFrame(client.run('?[id_e,ln] := *edge_label[id_e,ln]'))
df = edge_label[(edge_label["id_e"] >= 1) &(edge_label["id_e"] <= 500_000)]
cozo_rules = generate_edge_prop_cozo(df, batch_size=1000)

# Envoi des batchs
for rule in cozo_rules:
    print(client.run(rule))  



  status
0     OK


In [ ]:
# check
res = client.run('?[ id_e, k , v] := *edge_prop[ id_e, k, v]')
res